In [1]:
# Imports
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

In [2]:
# Set the device to 'mps' if CUDA is available, otherwise set it to 'cpu' for using apple silicon
device = "mps" if torch.backends.mps.is_available() else "cpu"
# Print the device that will be used for computations
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

mps


In [3]:
# Open the file 'wizard_of_oz.txt' in read mode with utf-8 encoding
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    # Read the entire content of the file into the variable 'text'
    text = f.read()

# Create a sorted set of unique characters present in the 'text'
chars = sorted(set(text))

# Calculate the size of the vocabulary, which is the number of unique characters in the 'text'
vocab_size = len(chars)

In [4]:
# Create a dictionary to map each character to a unique integer
string_to_int = { ch:i for i, ch in enumerate(chars) }
# Create a dictionary to map each integer back to its corresponding character
int_to_string = { i:ch for i, ch in enumerate(chars) }
# Define a function to encode a string into a list of integers
encode = lambda s: [string_to_int[c] for c in s]
# Define a function to decode a list of integers back into a string
decode = lambda l: ''.join([int_to_string[i] for i in l])
# Encode the entire text into a tensor of integers
data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
# Calculate the number of elements that correspond to 80% of the total data length
n = int(0.8*len(data))

# Split the data into training and validation sets
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[54, 67,  1, 76, 54, 65, 64, 62],
        [71, 58, 54, 73,  1, 60, 65, 54],
        [54, 67, 57,  0, 72, 73, 71, 54],
        [74, 65, 57,  1, 67, 58, 75, 58]], device='mps:0')
targets:
tensor([[67,  1, 76, 54, 65, 64, 62, 67],
        [58, 54, 73,  1, 60, 65, 54, 72],
        [67, 57,  0, 72, 73, 71, 54, 67],
        [65, 57,  1, 67, 58, 75, 58, 71]], device='mps:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
# # Set the block size to 8. This is the number of elements to consider at a time from the training data.
# block_size = 8

# # Split the training data into two parts: 'x' and 'y'. 'x' is the first 'block_size' elements and 'y' is the next 'block_size' elements.
# x, y = train_data[:block_size],  train_data[1:block_size+1]

# # Loop over the range of block_size
# for t in range(block_size):
#     # The context is the first 't+1' elements of 'x'
#     context = x[:t+1]
#     # The target is the 't'-th element of 'y'
#     target = y[t]
#     # Print the context and target
#     print('when input is', context, 'target is', target)

In [8]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

In [9]:
model = BigramLanguageModel(vocab_size) if not os.path.exists("wizard_of_oz.pth") else torch.load("wizard_of_oz.pth")
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"
o stthal Buskerthuigrturlllll cargibe o ait w chacitriceamact, tyo thaweme y-pruderenithed oucinvilsw d d watothainve wn Veg'thenoupre Wioule gan ty ha inonetinge I Jinopsesthe
"Nodvourthe terd abafinflitand we thove bialll bem tuly fe OFV*-dr re rd aghey,"Lan bbu agghoug Gasthemed orangso,"bllard squpisced Zesaidas nd he?"N moft bemepore. anth tspa pizaboooind n wll be p asestof I ff r
Wht ty here cke he "Ohad d weme avea telitooundeis oulerersutos frsost
ssincivematy d thedfl. bunthy. ougane


In [10]:
%%time
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 2.441, val loss: 2.454
step: 250, train loss: 2.414, val loss: 2.475
step: 500, train loss: 2.426, val loss: 2.460
step: 750, train loss: 2.428, val loss: 2.455
step: 1000, train loss: 2.434, val loss: 2.490
step: 1250, train loss: 2.448, val loss: 2.477
step: 1500, train loss: 2.408, val loss: 2.471
step: 1750, train loss: 2.434, val loss: 2.474
step: 2000, train loss: 2.425, val loss: 2.482
step: 2250, train loss: 2.428, val loss: 2.463
step: 2500, train loss: 2.424, val loss: 2.489
step: 2750, train loss: 2.440, val loss: 2.479
step: 3000, train loss: 2.429, val loss: 2.484
step: 3250, train loss: 2.429, val loss: 2.492
step: 3500, train loss: 2.418, val loss: 2.464
step: 3750, train loss: 2.417, val loss: 2.486
step: 4000, train loss: 2.427, val loss: 2.493
step: 4250, train loss: 2.428, val loss: 2.491
step: 4500, train loss: 2.462, val loss: 2.488
step: 4750, train loss: 2.423, val loss: 2.457
step: 5000, train loss: 2.428, val loss: 2.463
step: 5250, train l

In [11]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ARGaleckifomingoralemacofrde.
"Ve



[. ined. o y Jithe. je s wenklercham. toagghondantofonganlorble ilouler toteraimetil mar w, Hu ingllis s m whe, t, tadithebevespleke lerd. Wie s wanouley.
"Om.

in s thenaropef f oouse outerey sid, wecaclif ckeemapling fresthyofawiors y wosisher e boy R[Thutherf ancocedes hr tokifend twe cooowatofimod ylel cer, e sereceve as,"EWhaneance thee  ber wh. EThethy

thet Doo hinged we at icomay u n.
wd neal thad ay, t pe Gwiselysthee. hy c af oug s f choror po?"TR M


In [ ]:
torch.save(m, "wizard_of_oz.pth")